## AC Transit GTFS RT - Real-Time Vehicle Locations

This notebook calls the AC Transit API to collect real-time vehicle information and locations

**Links**: 
- [AC Transit API Documentation](https://api.actransit.org/transit/Help)
- [Using google.transit package to decode the protocol buffer returned by the API](https://github.com/MobilityData/gtfs-realtime-bindings/tree/master/python)


### Import packages

In [2]:
import pandas as pd
import numpy as np
import csv
import json
from datetime import datetime

import requests
from google.transit import gtfs_realtime_pb2

In [3]:
import os.path
from os import path

In [4]:
import schedule
import time
import functools

In [5]:
from config import *
BASE = "https://api.actransit.org/transit/"

### Functions to call API and save data to .csv file

In [6]:
def with_logging(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('LOG: Running job "%s" at %s' % (func.__name__, datetime.now().strftime("%m/%d, %H:%M:%S")))
        result = func(*args, **kwargs)
        print('LOG: Job "%s" completed at %s' % (func.__name__, datetime.now().strftime("%m/%d, %H:%M:%S")))
        return result
    return wrapper

In [7]:
#@with_logging
def call_api():
    
    # check to make sure between hours of operation
    dt_start = datetime.now()
    if (dt_start.weekday() < 5) & (dt_start.hour not in [1,2,3,4]):
        print("Calling API at %s" % dt_start.strftime("%m/%d, %H:%M:%S"))
    else:
        print("Skip call, out of hours (%s)" % dt_start.strftime("%m/%d, %H:%M:%S"))
        return
    
    # call API
    feed = gtfs_realtime_pb2.FeedMessage()
    response = requests.get(BASE+"gtfsrt/vehicles/?token="+KEY)
    feed.ParseFromString(response.content)
    
    # create empty arrays to store data
    route_id = []
    trip_id = []
    vehicle_id = []
    latitude = []
    longitude = []
    bearing = []
    speed = []
    unix_time = []
    #schedule_relationship = []

    # fill arrays with data pulled from API
    for entity in feed.entity:
        route_id.append(entity.vehicle.trip.route_id)
        trip_id.append(entity.vehicle.trip.trip_id)
        vehicle_id.append(entity.vehicle.vehicle.id)
        latitude.append(entity.vehicle.position.latitude)
        longitude.append(entity.vehicle.position.longitude)
        bearing.append(entity.vehicle.position.bearing)
        speed.append(entity.vehicle.position.speed)
        unix_time.append(entity.vehicle.timestamp)

    # save data to pandas dataframe
    df = pd.DataFrame(
        {'route_id' : route_id,
         'trip_id' : trip_id,
         'vehicle_id' : vehicle_id,
         'latitude' : latitude,
         'longitude' : longitude,
         'bearing' : bearing,
         'speed': speed,
         'unix_time': unix_time
        })
    df['time'] = pd.to_datetime(df['unix_time'], unit='s').astype('datetime64[ns, US/Pacific]')
    
    # save dataframe to CSV
    dt_end = datetime.now()
    out_file = "../data/ACTransit_Veh_Loc_"+dt_end.strftime("%m%d")+".csv"
    if path.exists(out_file):
        df.to_csv(out_file, mode='a', header=False, index=False)
    else:
        df.to_csv(out_file, mode='w', header=df.columns, index=False)
    
    # return feed, df
    print("Completed API call at %s" % dt_end.strftime("%m/%d, %H:%M:%S"))

### Run API (STATIC)

In [50]:
# call API once, save data to file
call_api()

Skip call, out of hours (03/01, 17:11:49)


### Run API (SCHEDULED)

In [8]:
# Clear existing schedule (if any)
schedule.clear()

# Schedule call
schedule.every(1).minutes.do(call_api)

# Run
while True:
    schedule.run_pending()
    time.sleep(0.05)

Calling API at 03/03, 13:06:13
Completed API call at 03/03, 13:06:15
Calling API at 03/03, 13:07:15
Completed API call at 03/03, 13:07:16
Calling API at 03/03, 13:08:16
Completed API call at 03/03, 13:08:16
Calling API at 03/03, 13:09:16
Completed API call at 03/03, 13:09:16
Calling API at 03/03, 13:10:16
Completed API call at 03/03, 13:10:16
Calling API at 03/03, 13:11:17
Completed API call at 03/03, 13:11:17
Calling API at 03/03, 13:12:17
Completed API call at 03/03, 13:12:17
Calling API at 03/03, 13:13:17
Completed API call at 03/03, 13:13:17
Calling API at 03/03, 13:14:18
Completed API call at 03/03, 13:14:18
Calling API at 03/03, 13:15:18
Completed API call at 03/03, 13:15:18
Calling API at 03/03, 13:16:18
Completed API call at 03/03, 13:16:19
Calling API at 03/03, 13:17:19
Completed API call at 03/03, 13:17:19
Calling API at 03/03, 13:18:19
Completed API call at 03/03, 13:18:19
Calling API at 03/03, 13:19:20
Completed API call at 03/03, 13:19:20
Calling API at 03/03, 13:20:20
Com

KeyboardInterrupt: 

### TESTING

In [ ]:
# https://stackoverflow.com/questions/54451027/using-python-with-schedule-library-with-an-if-statement

def weekdayJob():
    schedule.every(1).minutes.do(call_api).tag("one_min")

def clearJob():
    schedule.clear("one_min")

schedule.every().monday.at("05:00").do(weekdayJob)
schedule.every().tuesday.at("01:00").do(clearJob)

schedule.every().tuesday.at("05:00").do(weekdayJob)
schedule.every().wednesday.at("01:00").do(clearJob)

schedule.every().wednesday.at("05:00").do(weekdayJob)
schedule.every().thursday.at("01:00").do(clearJob)

schedule.every().thursday.at("05:00").do(weekdayJob)
schedule.every().friday.at("01:00").do(clearJob)

schedule.every().friday.at("05:00").do(weekdayJob)
schedule.every().saturday.at("01:00").do(clearJob)

while True:
    schedule.run_pending()
    time.sleep(0.05)